# CLT vs PJ

In [6]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd

## Cálculos CLT

### Cálculos Mensais

In [7]:
salario_mensal_bruto=[1_903.98, 2_826.66, 3_115.0, 4_664.68, 6_400, 8_800, 9_200, 15_000, 25_000, 30_000, 12_000 ]


In [8]:
def mostrar_descontos_mensais(funcao):
    """
    Formata saída para mostrar descontos mensais em formato de tabela
    """
    results =  [(f'{salario:,.2f}', f'{funcao(salario):,.2f}', f'{funcao(salario) / salario:.2%}') for salario in salario_mensal_bruto]
    df = pd.DataFrame(results, columns=["Salário","Imposto Descontado","Porcentagem"])
    return df

In [9]:
def imposto_de_renda_mensal(salario_mensal:float) -> float:
    """
    Calcula imposto de renda PF de acordo com faixa do ano de 2019
    https://impostoderenda2019.net.br/tabela-imposto-de-renda-2019/
    
    """
    saldo = salario_mensal
    aliquotas_faixas = {0.275: 4_664.68, 0.225: 3_751.06, 0.15: 2_826.66, 0.075: 1_903.98, 0: 0}
    ir = 0
    desconto_simplificado = 0.20  # 20% de desconto simplificado
    for aliquota, faixa in aliquotas_faixas.items():
        delta = max(saldo - faixa, 0)
        ir += delta * aliquota * (1 - desconto_simplificado)
        saldo = min(saldo, faixa)
    return ir


Para o cálculo do imposto de renda, foi usada a [tabela desse site](https://impostoderenda2019.net.br/tabela-imposto-de-renda-2019/), que está reproduzida abaixo:

| Base de Cálculo | Alíquota |
| --------------- | -------- |
|`De R$1.903,99 até R$2.826,65` | 7,5% |
|`De R$2.826,66 até R$3.751,05` | 15% | 
|`De R$3.751,06 até R$4.664,68` | 22,5% | 
|`Acima de R$4.664,68` | 27,5% |

Em seguida exibimos o salário, quanto foi descontado e o percentual real retido pelo imposto considerando o desconto simplificado.

In [10]:
mostrar_descontos_mensais(imposto_de_renda_mensal)


Salário Imposto Descontado Porcentagem
0   1,903.98               0.00       0.00%
1   2,826.66              55.36       1.96%
2   3,115.00              89.96       2.89%
3   4,664.68             330.74       7.09%
4   9,000.00           1,284.51      14.27%
5  15,000.00           2,604.51      17.36%
6  25,000.00           4,804.51      19.22%
7  30,000.00           5,904.51      19.68%
8  12,000.00           1,944.51      16.20%

In [11]:
def fgts_mensal(salario_mensal: float)-> float:
    """
    Calcula fgts a ser pago de imposto pelo empregador em 1 ano. 
    Aliquota de 8% + 40% disso em caso de demissão com justa causa -> 11,2%
    
    
    :param salario_mensal: salário mensal bruto do empregado
    :return: float
    """
    return salario_mensal * 0.112


In [12]:
mostrar_descontos_mensais(fgts_mensal)


Salário Imposto Descontado Porcentagem
0   1,903.98             213.25      11.20%
1   2,826.66             316.59      11.20%
2   3,115.00             348.88      11.20%
3   4,664.68             522.44      11.20%
4   9,000.00           1,008.00      11.20%
5  15,000.00           1,680.00      11.20%
6  25,000.00           2,800.00      11.20%
7  30,000.00           3,360.00      11.20%
8  12,000.00           1,344.00      11.20%

In [13]:
def inss_empregado(salario_mensal):
    """
    Calcula inss para 2019
    :param salario_mensal: 
    :return: 
    """
    if salario_mensal < 1_751.81:
        return salario_mensal * 0.08
    elif salario_mensal < 2_919.72:
        return salario_mensal * 0.09
    
    teto = 5_839.45
    return min(salario_mensal, teto) * 0.11


In [14]:

mostrar_descontos_mensais(inss_empregado)

Salário Imposto Descontado Porcentagem
0   1,903.98             171.36       9.00%
1   2,826.66             254.40       9.00%
2   3,115.00             342.65      11.00%
3   4,664.68             513.11      11.00%
4   9,000.00             642.34       7.14%
5  15,000.00             642.34       4.28%
6  25,000.00             642.34       2.57%
7  30,000.00             642.34       2.14%
8  12,000.00             642.34       5.35%

### Cálculos Anuais

In [15]:
def imposto_de_renda_anual(salario):
    """
    Calcula imposto de renda anual considerando Décimo terceiro e terço de ferias
    :param salario: 
    :return: 
    """
    ir = imposto_de_renda_mensal(salario) * 11
    salario_de_ferias = salario + salario / 3
    ir += imposto_de_renda_mensal(salario_de_ferias - inss_empregado(salario_de_ferias)) / 0.80
    ir += imposto_de_renda_mensal(salario - inss_empregado(salario)) / 0.80  # Tributação exlusiva de décimo terceiro
    return ir


In [16]:
def mostrar_descontos_anuais(funcao):
    salarios = salario_mensal_bruto
    salarios_anuais = [s * 13 + s / 3 for s in salario_mensal_bruto]
    results = [
        (f'{salario:,.2f}', f'{salario_anual:,.2f}', f'{funcao(salario):,.2f}', f'{funcao(salario) / salario_anual:.2%}') for
        salario,  salario_anual in zip(salarios, salarios_anuais)]
    df = pd.DataFrame(results, columns=["Salário Mensal", "Salário Anual", "Imposto Descontado", "Porcentagem"])
    return df

In [17]:
mostrar_descontos_anuais(imposto_de_renda_anual)


Salário Mensal Salário Anual Imposto Descontado Porcentagem
0       1,903.98     25,386.40              30.46       0.12%
1       2,826.66     37,688.80             807.44       2.14%
2       3,115.00     41,533.33           1,254.38       3.02%
3       4,664.68     62,195.73           4,600.50       7.40%
4       9,000.00    120,000.00          17,812.61      14.84%
5      15,000.00    200,000.00          36,182.61      18.09%
6      25,000.00    333,333.33          66,799.28      20.04%
7      30,000.00    400,000.00          82,107.61      20.53%
8      12,000.00    160,000.00          26,997.61      16.87%

In [18]:
def inss_anual(salario):
    """
    Calcula valor total decontado do empregado anualmente
    :param salario: 
    :return: 
    """
    ONZE_MESES_MAIS_DECIMO_TERCEIRO = 12
    inss = inss_empregado(salario) * ONZE_MESES_MAIS_DECIMO_TERCEIRO
    salario_ferias = salario + salario / 3
    return inss + inss_empregado(salario_ferias)


In [19]:
mostrar_descontos_anuais(inss_anual)


Salário Mensal Salário Anual Imposto Descontado Porcentagem
0       1,903.98     25,386.40           2,284.78       9.00%
1       2,826.66     37,688.80           3,467.37       9.20%
2       3,115.00     41,533.33           4,568.67      11.00%
3       4,664.68     62,195.73           6,799.72      10.93%
4       9,000.00    120,000.00           8,350.41       6.96%
5      15,000.00    200,000.00           8,350.41       4.18%
6      25,000.00    333,333.33           8,350.41       2.51%
7      30,000.00    400,000.00           8,350.41       2.09%
8      12,000.00    160,000.00           8,350.41       5.22%

In [20]:
def impostos_anuais(salario):
    return imposto_de_renda_anual(salario)+inss_empregado(salario)

In [21]:
def resumo_empregado(salario):
    """
    Retorna tupla com (Salario Mensal, Salario Mensal liquido,Salario Anual Liquido, Salario Liquido +fgts,Salario Bruto Anual , Impostos, FGTS, 
    :param funcao: 
    :return: 
    """

    salario_anual_bruto = salario * 13 + salario / 3
    impostos = impostos_anuais(salario)
    fgts_anual = fgts_mensal(salario)
    fgts_anual = fgts_anual * 13 + fgts_anual / 3
    salario_mensal_liquido = salario - imposto_de_renda_mensal(salario) - fgts_mensal(salario) - inss_empregado(salario)
    salario_anual_menos_impostos = salario_anual_bruto - impostos
    return salario, salario_mensal_liquido, salario_anual_menos_impostos, salario_anual_menos_impostos + fgts_anual, salario_anual_bruto, impostos, fgts_anual,



In [22]:
result = [map(lambda v: f'{v:<15,.2f}', resumo_empregado(salario)) for salario in salario_mensal_bruto]
df = pd.DataFrame(result, columns=["Salário Mensal", "Salário Mensal Líquido","Salário Anual Líquido","Salário Liquido +FGTS", "Salário Bruto Anual", "Imposto Descontado", "FGTS"])
display(df)


Salário Mensal Salário Mensal Líquido Salário Anual Líquido  \
0  1,903.98               1,519.38              25,184.58         
1  2,826.66               2,200.31              36,626.96         
2  3,115.00               2,333.51              39,936.31         
3  4,664.68               3,298.38              57,082.12         
4  9,000.00               6,065.15              101,545.05        
5  15,000.00              10,073.15             163,175.05        
6  25,000.00              16,753.15             265,891.72        
7  30,000.00              20,093.15             317,250.05        
8  12,000.00              8,069.15              132,360.05        

  Salário Liquido +FGTS Salário Bruto Anual Imposto Descontado  \
0       28,027.85           25,386.40          201.82            
1       40,848.11           37,688.80          1,061.84          
2       44,588.04           41,533.33          1,597.03          
3       64,048.04           62,195.73          5,113.61          
4       114,985.05          120,000.00         18,454.95         
5       185,575.05          200,000.00         36,824.95         
6       303,225.05          333,333.33         67,441.62         
7       362,050.05          400,000.00         82,749.95         
8       150,280.05          160,000.00         27,639.95         

              FGTS  
0  2,843.28         
1  4,221.15         
2  4,651.73         
3  6,965.92         
4  13,440.00        
5  22,400.00        
6  37,333.33        
7  44,800.00        
8  17,920.00

In [23]:
faturamento_bruto = [(s+fgts_mensal(s)) * (13 + 1 / 3) for s in salario_mensal_bruto]



In [24]:
display(pd.DataFrame([f'{v:,.2f}' for v in faturamento_bruto], columns=["Faturamento Bruto"]))

Faturamento Bruto
0         28,229.68
1         41,909.95
2         46,185.07
3         69,161.66
4        133,440.00
5        222,400.00
6        370,666.67
7        444,800.00
8        177,920.00

# PJ SIMPLES ANEXO III

In [25]:
def custos_simples_anexo_iii(faturamento_anual, contador=89, prolabore=1_100) -> float:
    """
    Calcula custos de impostos para empresa simples de anexo III
    DAS, Prólabore e Contador
    
    Aliquotas 2019: https://www.contabilizei.com.br/contabilidade-online/anexo-3-simples-nacional/
    :param faturamento_anual: 
    :param contador: 
    :param prolabore: 
    :return: 
    """
    saldo = faturamento_anual
    aliquotas_faixas = {0.33: 3_600_000, 0.21: 1_800_000, 0.16: 720_000, 0.135: 360_000, 0.112: 180_000, 0.06: 0}
    imposto = 0
    for aliquota, faixa in aliquotas_faixas.items():
        delta = max(saldo - faixa, 0)
        imposto += delta * aliquota
        saldo = min(saldo, faixa)
    inss = prolabore * 0.11
    return imposto + 12 * (contador + inss)


Para o cálculo de impostos PJ foi utilizada a tabela [desse site](https://www.contabilizei.com.br/contabilidade-online/anexo-3-simples-nacional/) reproduzida abaixo

| Faixa   | Alíquota|  Valor a Deduzir  | Receita Bruta em 12 Meses         |
| ------- | ------- | ----------------- | --------------------------------- |
|1a Faixa |	6,00%   |       -            |   Até 180.000,00                  | 
|2a Faixa |	11,20%  |   9.360,00	    |   De 180.000,01 a 360.000,00      |
|3a Faixa |	13,50%  |   17.640,00	    |   De 360.000,01 a 720.000,00      |
|4a Faixa |	16,00%  |   35.640,00	    |   De 720.000,01 a 1.800.000,00    |
|5a Faixa |	21,00%  |   125.640,00	    |   De 1.800.000,01 a 3.600.000,00  | 
|6a Faixa |	33,00%  |   648.000,00	    |   De 3.600.000,01 a 4.800.000,00  |

In [26]:
def resumo_pj(faturamento):
    custo_anual = custos_simples_anexo_iii(faturamento)
    return faturamento / 12, (faturamento - custo_anual) / 12, faturamento - custo_anual, faturamento 



In [27]:
results = [map(lambda v: f'{v:<15,.2f}', resumo_pj(faturamento)) for faturamento in faturamento_bruto]
display(pd.DataFrame(results, columns=["Faturamento Mensal", "Lucro Mensal", "Lucro Anual", "Faturamento Anual"]))

Faturamento Mensal     Lucro Mensal      Lucro Anual Faturamento Anual
0    2,352.47         2,001.32         24,015.90         28,229.68      
1    3,492.50         3,072.95         36,875.35         41,909.95      
2    3,848.76         3,407.83         40,893.96         46,185.07      
3    5,763.47         5,207.66         62,491.96         69,161.66      
4    11,120.00        10,242.80        122,913.60        133,440.00     
5    18,533.33        17,027.60        204,331.20        222,400.00     
6    30,888.89        27,978.89        335,746.67        370,666.67     
7    37,066.67        33,322.67        399,872.00        444,800.00     
8    14,826.67        13,727.07        164,724.80        177,920.00

# Resultado

In [28]:
data = []
for salario, faturamento in zip(salario_mensal_bruto, faturamento_bruto):
    result = []
    result.append(f'{salario:,.2f}')
    clt = resumo_empregado(salario)[2]
    result.append(f'{clt :<15,.2f}')
    pj = resumo_pj(faturamento)[2]
    result.append(f'{pj :<15,.2f}')
    result.append(f'{pj - clt :<15,.2f}')
    data.append(result)
df = pd.DataFrame(data, columns=["Salário Mensal", "CLT", "PJ", "DIFF"])
display(df)


Salário Mensal              CLT               PJ             DIFF
0       1,903.98  25,184.58        24,015.90        -1,168.68      
1       2,826.66  36,626.96        36,875.35        248.39         
2       3,115.00  39,936.31        40,893.96        957.66         
3       4,664.68  57,082.12        62,491.96        5,409.83       
4       9,000.00  101,545.05       122,913.60       21,368.55      
5      15,000.00  163,175.05       204,331.20       41,156.15      
6      25,000.00  265,891.72       335,746.67       69,854.95      
7      30,000.00  317,250.05       399,872.00       82,621.95      
8      12,000.00  132,360.05       164,724.80       32,364.75

In [29]:
data = []
index = []
for salario, faturamento in zip(salario_mensal_bruto, faturamento_bruto):
    result = []
    index.append(f'R$ {salario:,.2f}')
    clt = resumo_empregado(salario)[2]
    result.append(clt)
    pj = resumo_pj(faturamento)[2]
    result.append(pj)
    result.append(pj - clt)
    data.append(result)
df = pd.DataFrame(data, columns=[ "CLT", "PJ", "DIFF"], index=index)

In [30]:
ax = df.plot.bar(figsize=(9.5,5));
vals = ax.get_yticks();
ax.set_yticklabels([f'R${x:,.2f}' for x in vals]);
ax.set_ylabel("Lucro Anual");
ax.set_xlabel("Salário Mensal");




In [31]:

def custos_simples_anexo_v(faturamento_anual, contador=89) -> float:
    """
    Calcula custos de impostos para empresa simples de anexo V
    manipulando pro labore para 28% DAS, Prólabore e Contador
    
    Aliquotas 2019: 
    :param faturamento_anual: 
    :param contador: 
    :param prolabore: 
    :return: imposto
    """
    saldo = faturamento_anual
    aliquotas_faixas = {0.33: 3_600_000, 0.21: 1_800_000, 0.16: 720_000, 0.135: 360_000, 0.112: 180_000, 0.06: 0}
    imposto = 0
    for aliquota, faixa in aliquotas_faixas.items():
        delta = max(saldo - faixa, 0)
        imposto += delta * aliquota
        saldo = min(saldo, faixa)
    prolabore= 0.28 * faturamento_anual/12
    prolabore = max(prolabore, 1000) # prolabore nao pode ser menor que salario minimo
    inss = prolabore * 0.11
    ir = imposto_de_renda_mensal(prolabore)
    return imposto + 12 * (contador + inss + ir)



In [32]:
def resumo_pj_v(faturamento):
    custo_anual = custos_simples_anexo_v(faturamento)
    return faturamento / 12, (faturamento - custo_anual) / 12, faturamento - custo_anual, faturamento 


In [33]:
results = [map(lambda v: f'{v:<15,.2f}', resumo_pj_v(faturamento)) for faturamento in faturamento_bruto]
display(pd.DataFrame(results, columns=["Faturamento Mensal", "Lucro Mensal", "Lucro Anual", "Faturamento Anual"]))


Faturamento Mensal     Lucro Mensal      Lucro Anual Faturamento Anual
0    2,352.47         2,012.32         24,147.90         28,229.68      
1    3,492.50         3,083.95         37,007.35         41,909.95      
2    3,848.76         3,410.29         40,923.46         46,185.07      
3    5,763.47         5,151.15         61,813.78         69,161.66      
4    11,120.00        9,931.51         119,178.12        133,440.00     
5    18,533.33        16,131.61        193,579.31        222,400.00     
6    30,888.89        25,941.24        311,294.94        370,666.67     
7    37,066.67        30,714.20        368,570.35        444,800.00     
8    14,826.67        13,153.04        157,836.52        177,920.00

In [34]:
data = []
for salario, faturamento in zip(salario_mensal_bruto, faturamento_bruto):
    result = []
    result.append(f'{salario:,.2f}')
    clt = resumo_empregado(salario)[2]
    result.append(f'{clt :<15,.2f}')
    pj = resumo_pj(faturamento)[2]
    result.append(f'{pj :<15,.2f}')
    result.append(f'{pj - clt :<15,.2f}')
    pj_v = resumo_pj_v(faturamento)[2]
    result.append(f'{pj_v :<15,.2f}')
    result.append(f'{pj_v - clt :<15,.2f}')
    data.append(result)
df = pd.DataFrame(data, columns=["Salário Mensal", "CLT", "PJ - III", "DIFF - III", "PJ - V", "DIFF - V"])
display(df)

Salário Mensal              CLT         PJ - III       DIFF - III  \
0       1,903.98  25,184.58        24,015.90        -1,168.68         
1       2,826.66  36,626.96        36,875.35        248.39            
2       3,115.00  39,936.31        40,893.96        957.66            
3       4,664.68  57,082.12        62,491.96        5,409.83          
4       9,000.00  101,545.05       122,913.60       21,368.55         
5      15,000.00  163,175.05       204,331.20       41,156.15         
6      25,000.00  265,891.72       335,746.67       69,854.95         
7      30,000.00  317,250.05       399,872.00       82,621.95         
8      12,000.00  132,360.05       164,724.80       32,364.75         

            PJ - V         DIFF - V  
0  24,147.90        -1,036.68        
1  37,007.35        380.39           
2  40,923.46        987.16           
3  61,813.78        4,731.66         
4  119,178.12       17,633.07        
5  193,579.31       30,404.26        
6  311,294.94       45,403.22        
7  368,570.35       51,320.30        
8  157,836.52       25,476.47